# Классификация изображений

 Цель этого проекта - первое практическое знакомство с нейросетями и применение простых приемов для улучшения результата работы собранной нейросети.  
 
 Краткая схема действий:
 
 1. Загрузка библиотек и данных  
 2. Предварительный анализ даннных
 3. Аугментация данных с использованием библиотеки Albumentations и подготовка генераторов данных
 4. Transfer Learning - построение нейросети с использованием в качестве основы предобученной на Imagenet сети EfficientNetB6
 5. Fine Tuning - последовательное включение в работу слоев сети EfficientNetB6 (0% - 50% -100%)
 6. Подбор Learning Rate вручную и через callbacks, подбор optimizer
 7. Увеличение размера подаваемых изображений на последнем этапе обучения сети для повышения точности
 8. Test Time Augmentation (TTA) для усреднения нескольких предиктов и исключения случайных ошибок

 Основой для проекта послужили ноутбуки  BaseLine и transfer-learning-keras-flowers-sf-dl-v2 из курса. 

In [ ]:
!nvidia-smi

In [ ]:
!pip install -U efficientnet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os
import math

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks as C
import efficientnet.tfkeras as efn

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
EPOCHS               = 5  # эпох на обучение
BATCH_SIZE           = 64 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 0.001
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')

for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
# Примеры изображений по классам
for category in range(10):
    plt.figure(figsize=(16, 4))

    cat_image = train_df[train_df['Category'] == category].sample(4)
    cat_image_paths = cat_image['Id'].values
    cat_image_cat = cat_image['Category'].values

    for index, path in enumerate(cat_image_paths):
        im = PIL.Image.open(PATH+f'train/{cat_image_cat[index]}/{path}')
        plt.subplot(1,4, index+1)
        plt.imshow(im)
        plt.title('Class: '+str(cat_image_cat[index]))
        plt.axis('off')
    plt.show()

Видим, что классы соответствуют разным моделям/маркам автомобилей: класс 1 - форд "Фокус", класс 8 - фольксваген "Пассат", остальные - 
это 8 разных моделей ВАЗ. Фото различаются по размерам, яркости, четкости, ракурсам, перспективе и тд.  
При аугментации можем использовать все эти преобразования.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

Для аугментации данных используем библиотеку Albumentations

In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

Собираем набор преобразований исходных изображений

In [ ]:
AUG = A.Compose([
    A.HorizontalFlip(p=0.25),
    A.ToGray(p=0.15),
#    A.ToSepia(p=0.25),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.35),
    A.ShiftScaleRotate(always_apply=False, p=0.5, shift_limit=(-0.06, 0.06), rotate_limit=(-10, 10), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
    A.GaussianBlur(p=0.05),
    A.HueSaturationValue(p=0.25),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, always_apply=False, p=0.25),
])

In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUG,
        validation_split=VAL_SPLIT
)
test_datagen = ImageDataAugmentor(rescale=1./255)

### Генерация данных

In [ ]:
# Завернем наши данные в генератор:
train_generator = train_datagen.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, seed=RANDOM_SEED,
        subset='training')        # set as training data
test_generator = train_datagen.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, seed=RANDOM_SEED,
        subset='validation')      # set as validation data
test_sub_generator = test_datagen.flow_from_dataframe( 
        dataframe=sample_submission,
        directory=PATH+'test_upload/',
        x_col="Id",
        y_col=None,
        shuffle=False,
        class_mode=None,
        seed=RANDOM_SEED,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

In [ ]:
'''train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5,
#    shear_range=0.15,
    zoom_range=[0.85,1.15],
    brightness_range=[0.5, 1.5],
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=False,
#    fill_mode="nearest",
)

test_datagen = ImageDataGenerator(rescale=1. / 255)'''

In [ ]:
'''train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)'''

# Построение модели

### Загружаем предобученную сеть EfficientNetB6:

In [ ]:
input_shape

In [ ]:
base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
base_model.summary()

In [ ]:
# Не тренируем базовую модель на первом этапе
base_model.trainable = False

Собираем нейросеть: к базовой EfficientNetB6 добавляем два плотных слоя, второй из них - непосредственно для классификации.  
Используем также вспомогательные Pooling и Dropout слои, а также BatchNormalization

In [ ]:
# Making Model
model=M.Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(L.GlobalAveragePooling2D())
#model.add(L.Dropout(0.25))
#model.add(L.Flatten())
model.add(L.Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(L.Dropout(0.25))     # or 0.5
model.add(L.Dense(CLASS_NUM, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# How many layers
print(len(model.layers))

In [ ]:
len(model.trainable_variables)

In [ ]:
# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

## Обучение модели

# Step 1

# No Power CNN

In [ ]:
LR = 0.001
EPOCHS = 5
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

Добавим ModelCheckpoint, чтобы сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель, и  
EarlyStopping на будущее

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True,verbose=1)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
callbacks_list = [checkpoint, earlystop]

Обучаем:

In [ ]:
history = model.fit(
                    train_generator,
                    steps_per_epoch = train_generator.samples//train_generator.batch_size,
#                    steps_per_epoch = len(train_generator),
                    validation_data = test_generator, 
                    validation_steps = test_generator.samples//test_generator.batch_size,
#                    validation_steps = len(test_generator),
                    epochs = EPOCHS,
                    callbacks = callbacks_list
                    )

In [ ]:
# Подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_step1.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
# Посмотрим точность на этом шаге
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
def plot_history(history):
    plt.figure(figsize=(10,5))
    #plt.style.use('dark_background')
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'g', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    #plt.figure()
    plt.figure(figsize=(10,5))
    #plt.style.use('dark_background')
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_history(history)

На этом этапе достигается точность порядка 0.70 

# Step 2

# Half Power CNN

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

Разморозим половину слоев базовой нейросети

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers)//2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
len(base_model.trainable_variables)

In [ ]:
# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
# Изменим гиперпараметры
LR=0.0001
EPOCHS = 10
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

Добавим LearningRateScheduler для управления LR в ходе обучения.  
Попробуем профиль с небольшим холмом, чтобы выйти из возможного локального минимума

In [ ]:
# LearningRateScheduler - HandMade
lr_list = [1e-03, 2e-03, 5e-03, 2e-03, 1e-03, 5e-04, 2.5e-04, 1e-04, 1e-04, 5e-05]
def schedule(epoch):
    return lr_list[epoch]

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
lr_scheduler = LearningRateScheduler(schedule, verbose=1)
callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [ ]:
# Обучаем
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
model.save('../working/model_step2.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plot_history(history)

На этом этапе достигается точность 0.940 - 0.945  
Затраты времени на первые два этапа - 100 минут

# Step 3

# Full Power CNN

Включаем базовую модель на 100% и уменьшаем BATCH_SIZE до 16, чтобы не перегрузить GPU.

In [ ]:
base_model.trainable = True

In [ ]:
BATCH_SIZE = 16
LR=0.00001
EPOCHS = 10

Переопределяем генератор данных, чтобы учесть изменение размера батча

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

Пробуем разные LearningRateScheduler из callbacks

In [ ]:
'''# LearningRateScheduler - Exponential Decay
# Define configuration parameters
start_lr = LR
exp_decay = 0.33

# Define the scheduling function
def schedule(epoch):
    def lr(epoch, start_lr, exp_decay):
        return start_lr * math.exp(-exp_decay*epoch)
    return lr(epoch, start_lr, exp_decay)'''

In [ ]:
'''# LearningRateScheduler - HandMade
lr_list = [5e-5, 5e-5, 7e-5, 8e-5, 3e-5, 2e-5, 1.5e-5, 1e-5, 4e-5, 2.5e-5, 2e-5, 1e-5, 1e-5, 5e-6, 5e-6]
def schedule(epoch):
    return lr_list[epoch]'''

In [ ]:
# LearningRateScheduler - One CLR
# LR растет от start_lr линейно rampup_epochs эпох до max_lr, затем 3 делает шага на max_lr, далее убывает по экспоненте не очень круто в сторону min_lr 
# Define configuration parameters
start_lr = 0.00001
min_lr = 0.000001
max_lr = 0.0001
rampup_epochs = 4
sustain_epochs = 2
exp_decay = 0.8

# Define the scheduling function
def schedule(epoch):
    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        if epoch < rampup_epochs:
            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        else:
            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
lr_scheduler = LearningRateScheduler(schedule, verbose=1)
callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [ ]:
#from tensorflow import keras
#model = keras.models.load_model("../working/last_model/")
#model = M.load_model("../working/last_model/")

In [ ]:
# Обучаем
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
model.save('../working/model_step3.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plot_history(history)

На этом этапе достигается точность порядка 0.960 - 0.965  
Использование LearningRateScheduler не дает заметного улучшения  
Затраты времени - 8 минут на эпоху, всего - 80 мин

# Step 4

# Big Images

Увеличиваем размер подаваемых изображений, приходится уменьшать batch и резко возрастает время обсчитывания одной эпохи

In [ ]:
EPOCHS               = 5
BATCH_SIZE           = 4 # уменьшаем batch ещё больше, иначе не влезет в память на GPU
LR                   = 1e-6

IMG_SIZE             = 512 # увеличиваем размер картинок
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
AUG = A.Compose([
    A.HorizontalFlip(p=0.25),
    A.ToGray(p=0.25),
#    A.ToSepia(p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    A.ShiftScaleRotate(always_apply=False, p=0.5, shift_limit=(-0.06, 0.06), rotate_limit=(-10, 10), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
#    A.GaussianBlur(p=0.05),
#    A.HueSaturationValue(p=0.25),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, always_apply=False, p=0.25),
])

In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUG,
        validation_split=VAL_SPLIT
)
test_datagen = ImageDataAugmentor(rescale=1./255)

In [ ]:
'''train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#    rotation_range = 5,
#    shear_range=0.15,
    zoom_range=[0.85,1.15],
#    brightness_range=[0.75, 1.25],
    width_shift_range=0.1,
#    height_shift_range=0.1,
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=False,
#    fill_mode="nearest",
)

test_datagen = ImageDataGenerator(rescale=1. / 255)'''

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',       
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') 

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') 

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)


In [ ]:
# загружаем модель заново на новых размерах фото
base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
model=M.Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(L.GlobalAveragePooling2D())
#model.add(L.Dropout(0.25))
#model.add(L.Flatten())
model.add(L.Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(L.Dropout(0.25))     # or 0.5
model.add(L.Dense(CLASS_NUM, activation='softmax'))

In [ ]:
# Загружаем модель, сохраненную в предыдущую сессию
#from tensorflow import keras
#model = keras.models.load_model("../input/modelstep4last/model_step4.hdf5")

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
#model.load_weights('best_model.hdf5')
#model.load_weights("../input/modelstep4last/model_step4.hdf5")

In [ ]:
model.summary()

In [ ]:
# Перегружаем callbacks, чтобы убрать lr_scheduler 
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
callbacks_list = [checkpoint, earlystop]

In [ ]:
# Обучаем
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
        )

In [ ]:
model.save('../working/model_step4.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
#model.load_weights('model_step4.hdf5')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plot_history(history)  

На этом этапе достигается точность порядка 0.970 - 0.975, но есть потенциал роста при увеличении кол-ва эпох  
Затраты времени - 40 минут на эпоху, всего - 200 мин

Завершены 4 примерно одинаковых по структуре этапа обучения нейросети, теперь проверим результат

# Submission with TTA

Test Time Augmentation (TTA) для усреднения нескольких предиктов и исключения случайных ошибок

In [ ]:
model.load_weights('best_model.hdf5')

In [ ]:
AUG = A.Compose([
    A.HorizontalFlip(p=0.25),
    A.ToGray(p=0.25),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.25),
    A.ShiftScaleRotate(always_apply=False, p=0.5, shift_limit=(-0.06, 0.06), rotate_limit=(-10, 10), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
    A.GaussianBlur(p=0.05),
    A.HueSaturationValue(p=0.25),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, always_apply=False, p=0.25),
])

In [ ]:
test_datagen = ImageDataAugmentor(rescale=1./255)

In [ ]:
'''test_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5,
    shear_range=0.15,
    zoom_range=[0.85,1.15],
    brightness_range=[0.75, 1.25],
    width_shift_range=0.1,
    height_shift_range=0.1,
#    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=True,
#    fill_mode="nearest",
)'''

In [ ]:
BATCH_SIZE = 4

In [ ]:
test_sub_generator = test_datagen.flow_from_dataframe( 
        dataframe=sample_submission,
        directory=PATH+'test_upload/',
        x_col="Id",
        y_col=None,
        shuffle=False,
        class_mode=None,
        seed=RANDOM_SEED,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

In [ ]:
tta_steps = 5
predictions = []

for i in range(tta_steps):
    preds = model.predict(test_sub_generator, verbose=1) 
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
predictions = np.argmax(pred, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]
filenames_with_dir=test_sub_generator.filenames

submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

# Предсказание без TTA

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

In [ ]:
# Clean PATH
#import shutil
#shutil.rmtree(PATH)

# Итоги:

* Использована библиотека аугментации изображений Albumentations
* Применен transfer learning с fine-tuning
* Испробованы разные подходы к управлению LR, различные optimizer
* Размер картинки и батч изменялись на разных этапах   
* Сделан TTA (Test Time Augmentation)
* Лучший достигнутый результат - 0.97558
* BaseLine и ноутбук из учебного курса упростили выполнение задания на 80%, позволив  
  сосредоточиться непосредственно на вычислительной части - обучении нейросети 